# 4. Custom Model
This guide will step through the process of implementing a custom model that can be used in NeuRec. It will show example code snippets that can help build a custom model, however, further development is needed to successfully complete the model.

The process of implementing a custom model involves two main steps: creating a model class; and, adding the model to NeuRec. The following sections discusses these two steps in detail.

## Creating a Model Class
This step involves creating a class in the **model** folder. This class must extend the **AbstractRecommender** class, also in the model folder, and implement the required functions, namely:

* \_\_init\_\_()
* build_graph()
* train_model()
* predict()

> The class may have additional functions on top of these required functions.

Additionally, the **properties** instance variable is required in the custom model.

An example of a shell custom model is shown below:

```python
from neurec.model.AbstractRecommender import AbstractRecommender

class Custom(AbstractRecommender):
    properties = []
    
    def __init__(self, **kwds):
        super().__init__(**kwds)
        pass
        
    def build_graph(self):
        pass
        
    def train_model(self):
        pass
    
    def predict(self):
        pass
```

The following sections will discuss each of the requirements in turn, starting with the **properties** instance variable.

### properties
This variable defines the list of properties that the model requires to run. For example:

```python
properties = [
    "learning_rate",
    "embedding_size",
    "learner",
    "topk",
    "epochs",
    "eps",
    "adv",
    "adver",
    "adv_epoch",
    "reg",
    "reg_adv",
    "batch_size",
    "verbose",
    "loss_function"
]
```

These properties will then be read from the properties file when the model is instantiated, by the \_\_init\_\_() function in the AbstractRecommender class, and made available in the variable **self.conf**.

### \_\_init\_\_()
This function can be used to initialise any settings that are required by the model. It must first call the parent \_\_init\_\_() function, which will load the properties, discussed above. Additionally, it will load the dataset into **self.dataset**.

```python
from neurec.evaluation import Evaluate
from neurec.model.AbstractRecommender import AbstractRecommender
from neurec.util import reader, learner
import numpy as np
import tensorflow as tf

class Custom(AbstractRecommender):
    def __init__(self):
        super().__init__(**kwds)
        
        # Load necessary settings from properties file
        self.layers = self.conf["layers"]
        self.learning_rate = self.conf["learning_rate"]
        self.learner = self.conf["learner"]
        self.loss_function = self.conf["loss_function"]
        self.num_epochs= self.conf["epochs"])
        self.batch_size= self.conf["batch_size"])
        # etc...
        
    # ...
```

> If you need to log to the console, you can use the **self.logger** variable, further discussed [here](https://docs.python.org/3.6/library/logging.html)

### build_graph()
The build_graph() function is called by NeuRec before training the model and, therefore, can be used to create the network. Here, you can setup the loss and optimiser functions for the model, which can then be used while training the model:

```python
from neurec.evaluation import Evaluate
from neurec.model.AbstractRecommender import AbstractRecommender
from neurec.util import reader, learner
import numpy as np
import tensorflow as tf

class Custom(AbstractRecommender):
    # ...
    
    def build_graph(self):
        self.loss = learner.pairwise_loss(self.loss_function,result) + self.reg_mf * ( tf.reduce_sum(tf.square(p1)) \
                + tf.reduce_sum(tf.square(q2)) + tf.reduce_sum(tf.square(q1)))
        
        self.optimizer = learner.optimizer(self.learner, self.loss, self.learning_rate)
        
    # ...
```

### train_model()
This function is called by NeuRec to train the model. You can use the data_gen module to generate the necessary data for training, which can be found in the **util** folder. Additionally, the Evaluate class can be used here to establish the performance of the model, in this example after every epoch.

```python
from neurec.evaluation import Evaluate
from neurec.model.AbstractRecommender import AbstractRecommender
from neurec.util import reader, learner
import numpy as np
import tensorflow as tf

class Custom(AbstractRecommender):
    # ...
    
    def train_model(self):
        for epoch in range(self.num_epochs):
            user_input, item_input_pos, item_input_neg = data_gen._get_pairwise_all_data(self.dataset)
            
            num_training_instances = len(user_input)
            
            for num_batch in np.arange(int(num_training_instances/self.batch_size)):
                bat_users, bat_items_pos, bat_items_neg = data_gen._get_pairwise_batch_data(user_input, item_input_pos, item_input_neg, num_batch, self.batch_size)
                
            Evaluate.test_model(self,self.dataset)
            
    # ...
```

### predict()
The predict() function is used by the Evaluate classes to establish the performance of the model. An example of a predict function is given below:

```python
from neurec.evaluation import Evaluate
from neurec.model.AbstractRecommender import AbstractRecommender
from neurec.util import reader, learner
import numpy as np
import tensorflow as tf

class Custom(AbstractRecommender):
    # ...
    
    def predict(self):
        users = np.full(len(items), user_id, dtype=np.int32)
        return self.sess.run(self.output, feed_dict={self.user_input: users, self.item_input: items})
    
    # ...
```

## Adding to NeuRec
After creating a custom model, NeuRec needs to be updated so that it knows the model is available. To do this, the model needs to be added to **data/models.py** as follows:

```python
import Custom

models = {
    "custom": Custom
}
```

> Make sure to set the properties file to use the new model, discussed in 3. Properties